In [1]:
# from sklearn.model_selection import GroupShuffleSplit
import pandas as pd  

In [2]:
import os
import os.path
from google.colab import drive

# mount Google Drive to /content/drive/My Drive/
if os.path.isdir("/content/drive/MyDrive/Web Mining"):
  print("Google Drive already mounted")
else:
  drive.mount('/content/drive',force_remount=True)  
os.chdir("/content/drive/MyDrive/Web Mining") 

Google Drive already mounted


In [6]:
# do everything with final test set now
cf_list = pd.read_csv('./Final Data/Single-CF-test.csv')
print(cf_list)
cf_list.describe()

        UserID  MovieID    Rating
0            2     1357  3.650502
1            2     3068  3.589013
2            2     1537  3.635573
3            2      647  3.565670
4            2     2194  3.860406
...        ...      ...       ...
209641    6038     1387  3.796086
209642    6038     2700  3.666036
209643    6038     2716  3.774065
209644    6038     3396  3.645155
209645    6038     1079  3.736976

[209646 rows x 3 columns]


,UserID,MovieID,Rating
count,209646.000000,209646.000000,209646.000000
mean,3083.464101,1872.295880,3.412923
std,1788.643743,1096.757168,0.212479
min,2.000000,1.000000,2.640933
25%,1451.000000,1029.000000,3.258420
50%,3089.000000,1883.000000,3.385139
75%,4704.000000,2788.000000,3.538141
max,6038.000000,3952.000000,4.267178


In [7]:
# cf_list.loc[cf_list['MovieID'] == 978]

In [8]:
users = pd.read_csv('./Abhay/users_grouped.csv')
print(users.head())
users = users.drop(columns=['Gender', 'Age', 'Occupation'])
print(users)

   UserID  Cluster Gender  Age  Occupation  Group
0    6040        0      M   25           6      1
1    3961        0      M   25           1      1
2    3966        0      M   25           0      1
3    3968        0      M   25           2      1
4    1835        0      M   25          19      1
      UserID  Cluster  Group
0       6040        0      1
1       3961        0      1
2       3966        0      1
3       3968        0      1
4       1835        0      1
...      ...      ...    ...
6035    3904       19    596
6036     940       19    596
6037    2217       19    596
6038    5063       19    596
6039       1       19    579

[6040 rows x 3 columns]


In [9]:
grouped = pd.merge(cf_list, users, on="UserID", how="left")

In [12]:
grouped.describe()

,UserID,MovieID,Rating,Cluster,Group
count,209646.000000,209646.000000,209646.000000,209646.000000,209646.000000
mean,3083.464101,1872.295880,3.412923,5.015808,263.740062
std,1788.643743,1096.757168,0.212479,5.528298,168.343882
min,2.000000,1.000000,2.640933,0.000000,3.000000
25%,1451.000000,1029.000000,3.258420,0.000000,111.000000
50%,3089.000000,1883.000000,3.385139,2.000000,239.000000
75%,4704.000000,2788.000000,3.538141,9.000000,389.000000
max,6038.000000,3952.000000,4.267178,19.000000,593.000000


In [13]:
grouped_ratings = pd.DataFrame(columns=['Group', 'MovieID', 'Rating'])
grouped_ratings

,Group,MovieID,Rating


In [14]:
movieid_list = cf_list['MovieID'].unique().tolist()
print((movieid_list))

[1357, 3068, 1537, 647, 2194, 648, 2268, 2628, 1103, 2916, 3468, 1210, 1792, 1687, 1213, 3578, 2881, 3030, 1217, 3105, 434, 2126, 3107, 3108, 3035, 1253, 1610, 292, 2236, 3071, 902, 368, 1259, 3147, 1544, 1293, 1188, 3255, 3256, 3257, 110, 2278, 2490, 1834, 3471, 589, 1690, 3654, 2852, 1945, 982, 1873, 2858, 1225, 2028, 515, 442, 2312, 265, 1408, 1084, 3699, 480, 1442, 2067, 1265, 1370, 1193, 1801, 1372, 2353, 3334, 2427, 590, 1196, 1552, 736, 1198, 593, 2359, 95, 2717, 2571, 1917, 2396, 3735, 1953, 1597, 3809, 1954, 1955, 235, 1124, 1957, 163, 21, 165, 2321, 1090, 380, 2501, 349, 457, 1096, 920, 459, 1527, 3418, 1385, 3451, 3095, 780, 498, 2728, 2002, 1962, 1784, 2943, 2006, 318, 1207, 1968, 3678, 1244, 356, 1245, 1246, 3893, 1247, 2622, 3358, 3359, 1682, 1756, 1320, 2124, 2125, 1250, 2054, 1252, 720, 3868, 1254, 3869, 1256, 3500, 1257, 3501, 2997, 653, 1831, 3363, 586, 587, 3438, 588, 3439, 3296, 223, 150, 2496, 1, 2497, 2, 2498, 153, 7, 2133, 2135, 3948, 2136, 2137, 802, 2138, 1409,

In [15]:
movieratings_list = [None] * len(movieid_list)

In [48]:
def aggregation(method): 
  if (method == 'average'): 
    test = grouped.groupby(by=['Group', 'MovieID'], as_index=False)['Rating'].mean() 
    test2 = users.groupby('Group')['UserID'].agg('count').reset_index()
    final = pd.merge(test, test2, on=["Group"], how="left")
    final = final.rename(columns = {'UserID':'UserCount'})
    # final['Rating'] = final.apply(lambda x: x['Rating'] / x['UserCount'], axis=1) 
    return final
  elif (method == 'least_misery'): 
    test = grouped.groupby(by=['Group', 'MovieID'], as_index=False)['Rating'].min() 
    test2 = users.groupby('Group')['UserID'].agg('count').reset_index()
    final = pd.merge(test, test2, on=["Group"], how="left")
    final = final.rename(columns = {'UserID':'UserCount'})
    return final
  elif (method == 'most_pleasure'): 
    test = grouped.groupby(by=['Group', 'MovieID'], as_index=False)['Rating'].max() 
    test2 = users.groupby('Group')['UserID'].agg('count').reset_index()
    final = pd.merge(test, test2, on=["Group"], how="left")
    final = final.rename(columns = {'UserID':'UserCount'})
    return final
  elif (method == 'average_without_misery'): 
    test = grouped.copy()
    test = test[test['Rating'] > 2.5]
    test = test.groupby(by=['Group', 'MovieID'], as_index=False)['Rating'].max() 
    test2 = users.groupby('Group')['UserID'].agg('count').reset_index()
    final = pd.merge(test, test2, on=["Group"], how="left")
    final = final.rename(columns = {'UserID':'UserCount'})
    return final

In [72]:
import os
def convert_df_to_csv(dataframe, method): 
  path='./Final Data'

  dataframe['Ranking'] = dataframe.groupby(by=['Group'], as_index=False)['Rating'].rank("dense", ascending=False)
  dataframe = pd.merge(grouped, dataframe, on=['Group', 'MovieID'], how='left')
  dataframe = dataframe.drop(['Rating_y', 'UserCount', 'Ranking', 'Cluster', 'Group'], axis=1)
  dataframe = dataframe.rename(columns = {'Rating_x':'Rating'})
  name = 'user_ratings_cf_' + method
  name = name + '.csv' 
  output_file = os.path.join(path, name)
  dataframe.to_csv(output_file, index=False)

In [78]:
ave_df = aggregation('average')
convert_df_to_csv(ave_df, 'average')

In [73]:
least_misery_df = aggregation('least_misery')
convert_df_to_csv(least_misery_df, 'least_misery')

In [74]:
most_pleasure_df = aggregation('most_pleasure')
convert_df_to_csv(most_pleasure_df, 'most_pleasure')

In [75]:
average_without_misery_df = aggregation('average_without_misery')
convert_df_to_csv(average_without_misery_df, 'average_without_misery')

In [76]:
test = pd.read_csv('./Final Data/user_ratings_cf_average_without_misery.csv', index_col=False)

In [77]:
print(test)

        UserID  MovieID    Rating
0            2     1357  3.650502
1            2     3068  3.589013
2            2     1537  3.635573
3            2      647  3.565670
4            2     2194  3.860406
...        ...      ...       ...
209641    6038     1387  3.796086
209642    6038     2700  3.666036
209643    6038     2716  3.774065
209644    6038     3396  3.645155
209645    6038     1079  3.736976

[209646 rows x 3 columns]


In [54]:
test.describe()

,Group,MovieID,Rating,Ranking,UserCount
count,126346.000000,126346.000000,126346.000000,126346.000000,126346.000000
mean,267.660876,1925.600423,3.387818,577.740601,10.125861
std,169.093013,1110.026720,0.196198,389.165567,0.331694
min,3.000000,1.000000,2.718617,1.000000,10.000000
25%,119.000000,1035.000000,3.246734,264.000000,10.000000
50%,249.000000,1964.000000,3.365967,527.000000,10.000000
75%,395.000000,2861.000000,3.504666,825.000000,10.000000
max,593.000000,3952.000000,4.267178,2198.000000,11.000000


In [59]:
test['Rating'].count()

126346

In [57]:
users

,UserID,Cluster,Group
0,6040,0,1
1,3961,0,1
2,3966,0,1
3,3968,0,1
4,1835,0,1
...,...,...,...
6035,3904,19,596
6036,940,19,596
6037,2217,19,596
6038,5063,19,596
